In [1]:
import sparknlp

spark = sparknlp.start()

24/11/12 16:33:56 WARN Utils: Your hostname, ubuntu-IdeaPad-Gaming-3-15IAH7 resolves to a loopback address: 127.0.1.1; using 192.168.1.16 instead (on interface enp48s0)
24/11/12 16:33:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntu/.local/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0cb33f2c-5c6b-4362-a3ac-3066988f09e8;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.5.1 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in centra

In [24]:
from sparknlp.annotator import (
    Tokenizer, BertForZeroShotClassification
    )
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler

document_assembler = DocumentAssembler() \
.setInputCol('text') \
.setOutputCol('document')

tokenizer = Tokenizer() \
.setInputCols(['document']) \
.setOutputCol('token')

zeroShotClassifier = BertForZeroShotClassification \
.pretrained('bert_base_cased_zero_shot_classifier_xnli', 'en') \
.setInputCols(['token', 'document']) \
.setOutputCol('class') \
.setCaseSensitive(True) \
.setMaxSentenceLength(512) \
.setCandidateLabels(["Credit reporting", "Debt collection", "Mortgage", "Student loan", "Credit card", "Checking or savings account", "Money transfer", "Vehicle loan or lease", "Personal loan", "Fraud theft and scam", "Other"])

pipeline = Pipeline(stages=[
document_assembler,
tokenizer,
zeroShotClassifier
])

bert_base_cased_zero_shot_classifier_xnli download started this may take some time.


24/11/12 17:09:46 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


Approximate size to download 387.7 MB
[OK!]


In [28]:
example = spark.createDataFrame([['To Whom It May Concern, Subject : Formal Complaint, Opt-Out of Inaccurate Reporting, and Demand for Immediate Updates I am writing to file a formal complaint against the Credit Reporting Agencies ( CRAs ) regarding the inaccurate reporting of my credit information. As a federally protected consumer under the Fair Credit Reporting Act ( FCRA ), specifically 15 U.S.C. 1681 et seq., I am exercising my rights concerning the privacy of my personal information and the accuracy of my credit report. The FCRA mandates that CRAs must follow reasonable procedures to protect the confidentiality of credit information and promote accuracy, fairness, and the privacy of personal information. Notably, 15 U.S.C. 1681a states that it is illegal to report transaction history on a credit report, as the Act excludes reports that contain information solely about transactions or experiences between the consumer and the reporting entity. Given the inaccuracies in my credit report, I am opting out of any and all authorization for the reporting or sharing of any information related to the accounts inaccurately reported. This includes any written, unwritten, verbal, or nonverbal authorization I may have provided, in accordance with 15 U.S.C. 6802 and 16 CFR Part 313, which govern the Privacy of Consumer Financial Information. Due to the violations of my rights under the FCRA and the impact on my creditworthiness, I am demanding immediate updates to my credit report to correct these inaccuracies. I expect these updates to be made as soon as possible, in compliance with the FCRAs accuracy standards. Should these updates not be made, I will be seeking compensation for the violation of my rights as outlined under 15 U.S.C. 6801-6805, 16 CFR Part 313, and 15 U.S.C. 1692k. I am prepared to pursue legal remedies should my demands not be met. To resolve this matter, I require the following : Immediate correction of the inaccurately reported account ( s ) on my credit report to comply with the FCRAs standards. Compensation for the violation of my rights under the FCRA and related statutes due to the mishandling of my personal information. Written confirmation of the actions taken to address my requests as required under the FCRA. Failure to address these concerns will compel me to take further legal action to protect my rights and seek full compensation for the damages incurred. Thank you for your prompt attention to this serious matter. Company : Kikoff Lending LLC']]).toDF("text")
from pyspark.sql.functions import substring

# Limitar el texto a los primeros 512 caracteres (aproximadamente)
example = example.withColumn("text", substring("text", 1, 512))
result = pipeline.fit(example).transform(example)
result.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

To Whom It May Concern, Subject : Formal Complaint, Opt-Out of Inaccurate Reporting, and Demand for Immediate Updates I am writing to file a formal complaint against the Credit Reporting Agencies ( CRAs ) regarding the inaccurate reporting of my credit information. As a federally protected consumer under the Fair Credit Reporting Act ( FCRA ), specifically 15 U.S.C. 1681 et seq., I am exercising my rights concerning the privacy of my personal information and the accuracy of my credit report. The FCRA mandat